In [8]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [9]:
dataDir='/Users/azakaria/Downloads'
dataType='val2017'
annFile=f'{dataDir}/annotations/instances_{dataType}.json'

In [10]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.80s)
creating index...
index created!


In [11]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
#print(len(cats))
#print(coco.anns.keys()) #keys are the image ids I believe
#print(coco.anns[1768].keys())
nms=[cat['name'] for cat in cats]
#print(nms)
#print([cat['id'] for cat in cats])
print(cat_map)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 13: 11, 14: 12, 15: 13, 16: 14, 17: 15, 18: 16, 19: 17, 20: 18, 21: 19, 22: 20, 23: 21, 24: 22, 25: 23, 27: 24, 28: 25, 31: 26, 32: 27, 33: 28, 34: 29, 35: 30, 36: 31, 37: 32, 38: 33, 39: 34, 40: 35, 41: 36, 42: 37, 43: 38, 44: 39, 46: 40, 47: 41, 48: 42, 49: 43, 50: 44, 51: 45, 52: 46, 53: 47, 54: 48, 55: 49, 56: 50, 57: 51, 58: 52, 59: 53, 60: 54, 61: 55, 62: 56, 63: 57, 64: 58, 65: 59, 67: 60, 70: 61, 72: 62, 73: 63, 74: 64, 75: 65, 76: 66, 77: 67, 78: 68, 79: 69, 80: 70, 81: 71, 82: 72, 84: 73, 85: 74, 86: 75, 87: 76, 88: 77, 89: 78, 90: 79}


In [17]:
catIds = coco.getCatIds()
def get_imgs_from_each_category(catIds, numImgs):
    imgIds = []
    for catId in catIds:
        imgIds_current_batch = coco.getImgIds(catIds=[catId])[:]
        if(len(imgIds_current_batch) < numImgs):
            print(f'category {catId}: {len(imgIds_current_batch)} total images. WARNING! Less than {numImgs} images threshold')
            #print(f'WARNING! Less than {numImgs} images in category {catId}! {len(imgIds_current_batch)} total images')
        else:
            print(f'category {catId}: {len(imgIds_current_batch)} total images')
        imgIds = imgIds + imgIds_current_batch
    return imgIds

In [13]:
from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
ann_c = set()
def write_label_file(imgId):
    # using with statement
    img = coco.loadImgs(imgId)[0]
    label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/coco/dataset/labels')
    filename = img['file_name'].split('.')[0] + '.txt'
    #print(img['file_name'])
    with open(label_dir/filename, 'w') as file:
        for ann in coco.loadAnns(coco.getAnnIds(imgIds = imgId, catIds=catIds)):
            ann_c.add(cat_map[ann["category_id"]])
            if cat_map[ann["category_id"]] > 80:
                print(f'cat_id > 80 {ann["category_id"]}')
            x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
            file.write(f'{cat_map[ann["category_id"]]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
    return

In [18]:
img_ids = get_imgs_from_each_category(catIds, 120)
"""
for img_id in img_ids:
    write_label_file(img_id)
"""

#print(len(ann_c))
#print(ann_c)

category 1: 2693 total images
category 2: 149 total images
category 3: 535 total images
category 4: 159 total images
category 5: 97 total images. WARNING! Less than 120 images threshold
category 6: 189 total images
category 7: 157 total images
category 8: 250 total images
category 9: 121 total images
category 10: 191 total images
category 11: 86 total images. WARNING! Less than 120 images threshold
category 13: 69 total images. WARNING! Less than 120 images threshold
category 14: 37 total images. WARNING! Less than 120 images threshold
category 15: 235 total images
category 16: 125 total images
category 17: 184 total images
category 18: 177 total images
category 19: 128 total images
category 20: 65 total images. WARNING! Less than 120 images threshold
category 21: 87 total images. WARNING! Less than 120 images threshold
category 22: 89 total images. WARNING! Less than 120 images threshold
category 23: 49 total images. WARNING! Less than 120 images threshold
category 24: 85 total images

'\nfor img_id in img_ids:\n    write_label_file(img_id)\n'

In [ ]:
import os
import shutil

def copy_images_for_dataset_dir():
    src_image_dir = Path('/Users/azakaria/Downloads/val2017')
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + '.jpg'
        dest = '/Users/azakaria/Code/openmpf-yolo-training/dataset/images/' + f.split('.')[0] + '.jpg'
        print(str(src_image_dir/f.split('.')[0]) + '.jpg')
        shutil.copyfile(src, dest)
        #print(image_dir/f.split('.')[0])
    return
copy_images_for_dataset_dir()